# Training a tweet generator model


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [3]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
#    test_dataset = TextDataset(
#          tokenizer=tokenizer,
#          file_path=test_path,
#          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset ,data_collator

train_dataset ,data_collator = load_dataset('tweets_train.txt', tokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (113470 > 1024). Running this sequence through the model will result in indexing errors


In [4]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-tweetgen", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [5]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=84, training_loss=3.549079168410528, metrics={'train_runtime': 97.5001, 'train_samples_per_second': 0.862, 'total_flos': 254024455421952, 'epoch': 3.0})

In [6]:
trainer.save_model()

In [10]:
from transformers import pipeline

generator = pipeline('text-generation',model='./gpt2-tweetgen', tokenizer='gpt2',config={'max_length':280})


In [12]:
generator('America')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "America. (Don't forget about Trump's tweet!)"}]

In [14]:
generator('Me')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Me on. We are all very, very sad to hear about what has been done to this family after so many years. We want to thank those who were here to help us, like our president's team, our state officials, to fight for"}]

In [15]:
generator('Bernie')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Bernie and Senator Sanders today joined us today in calling for Donald Rumsfeld to be impeached for lying to Vice President Joe Biden about his support for war in Iraq.\n\nLet me come to you now and discuss what is at stake for the'}]

In [16]:
generator('')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The next wave, as Donald Trump has vowed, will target American workers. A few small groups are trying for change: young workers, minorities and people of color.\n\nThe American Dream deserves a president who will make fair and free trade a'}]